<a href="https://colab.research.google.com/github/Karthikard/ML/blob/main/27_pca.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from google.colab import files
uploaded = files.upload()

Saving 27_heart.xlsx to 27_heart.xlsx


In [4]:
import pandas as pd
df = pd.read_excel('27_heart.xlsx')
df.head()

,Age,Sex,ChestPainType,RestingBP,Cholesterol,FastingBS,RestingECG,MaxHR,ExerciseAngina,Oldpeak,ST_Slope,HeartDisease
0,40,M,ATA,140,289,0,Normal,172,N,0.0,Up,0
1,49,F,NAP,160,180,0,Normal,156,N,1.0,Flat,1
2,37,M,ATA,130,283,0,ST,98,N,0.0,Up,0
3,48,F,ASY,138,214,0,Normal,108,Y,1.5,Flat,1
4,54,M,NAP,150,195,0,Normal,122,N,0.0,Up,0


In [5]:
df = pd.get_dummies(df,drop_first='True').astype(int)
df.head()

,Age,RestingBP,Cholesterol,FastingBS,MaxHR,Oldpeak,HeartDisease,Sex_M,ChestPainType_ATA,ChestPainType_NAP,ChestPainType_TA,RestingECG_Normal,RestingECG_ST,ExerciseAngina_Y,ST_Slope_Flat,ST_Slope_Up
0,40,140,289,0,172,0,0,1,1,0,0,1,0,0,0,1
1,49,160,180,0,156,1,1,0,0,1,0,1,0,0,1,0
2,37,130,283,0,98,0,0,1,1,0,0,0,1,0,0,1
3,48,138,214,0,108,1,1,0,0,0,0,1,0,1,1,0
4,54,150,195,0,122,0,0,1,0,1,0,1,0,0,0,1


In [9]:
input = df.drop('HeartDisease', axis=1)
output = df['HeartDisease']

In [10]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaled = scaler.fit_transform(input)
scaled

array([[-1.4331398 ,  0.41090889,  0.82507026, ..., -0.8235563 ,
        -1.00218103,  1.15067399],
       [-0.47848359,  1.49175234, -0.17196105, ..., -0.8235563 ,
         0.99782372, -0.86905588],
       [-1.75135854, -0.12951283,  0.7701878 , ..., -0.8235563 ,
        -1.00218103,  1.15067399],
       ...,
       [ 0.37009972, -0.12951283, -0.62016778, ...,  1.21424608,
         0.99782372, -0.86905588],
       [ 0.37009972, -0.12951283,  0.34027522, ..., -0.8235563 ,
         0.99782372, -0.86905588],
       [-1.64528563,  0.30282455, -0.21769643, ..., -0.8235563 ,
        -1.00218103,  1.15067399]])

In [11]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(scaled, output, test_size=0.2, random_state=30)

Classification models

In [12]:
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV

In [13]:
model_params = {
    'svm' :
    {
        'model' : SVC(gamma = 'auto'),
        'params' :
        {
            'C' : [1,10,20],
            'kernel' : ['rbf', 'linear']
        }
    },

    'lr' :
    {
        'model' : LogisticRegression(max_iter=5000),
        'params' :
        {
            'C' : [1,5,10],
            'solver' : ['liblinear', 'lbfgs']
        }
    },

    'dt' :
    {
        'model' : DecisionTreeClassifier(),
        'params' :
        {
            'criterion' : ['gini', 'entropy']
        }
    },

    'rf' :
    {
        'model' : RandomForestClassifier(),
        'params' :
        {
            'n_estimators' : [10,50,100]
        }
    }

}

In [29]:
scores = []

for model_name in model_params:
    clf = GridSearchCV(model_params[model_name]['model'], model_params[model_name]['params'], cv=5, return_train_score=False)
    clf.fit(x_train, y_train)
    scores.append({
        'model': model_name,
        'best_score': clf.best_score_,
        'best_params': clf.best_params_
    })

In [30]:
import pandas as pd
df = pd.DataFrame(scores, columns=['model', 'best_score', 'best_params'])
df

,model,best_score,best_params
0,svm,0.869239,"{'C': 1, 'kernel': 'rbf'}"
1,lr,0.858345,"{'C': 1, 'solver': 'liblinear'}"
2,dt,0.783403,{'criterion': 'entropy'}
3,rf,0.858336,{'n_estimators': 100}


In [16]:
from sklearn.decomposition import PCA
pca = PCA(0.95)
x_pca = pca.fit_transform(scaled)
x_pca

array([[ 3.10322476e+00,  4.53348653e-01,  7.29354131e-01, ...,
        -2.81716211e-01, -2.21130833e-02,  2.99076030e-01],
       [ 1.88575560e-01,  1.09859531e+00,  7.04751815e-01, ...,
        -8.89134950e-01, -2.47695163e-01, -5.30215318e-02],
       [ 1.60169874e+00, -2.12049455e+00,  5.04457557e-01, ...,
         3.80592128e-01, -2.02703750e+00,  5.41199357e-01],
       ...,
       [-1.75381674e+00,  1.48385851e+00, -3.12917676e-01, ...,
         4.33233327e-02,  4.44874741e-01,  5.18144440e-02],
       [ 1.07029619e+00, -1.88973383e-01,  1.80807242e+00, ...,
        -1.17762577e+00,  1.40085670e+00,  2.07430932e-02],
       [ 2.49669350e+00,  4.73266866e-01, -1.01566593e+00, ...,
        -3.94855418e-02, -6.99462872e-02,  2.84979449e-03]])

In [17]:
x_pca.shape

(918, 13)

In [18]:
input.shape

(918, 15)

In [19]:
pca.explained_variance_ratio_

array([0.22449803, 0.11023583, 0.09442117, 0.08190894, 0.07459737,
       0.07093285, 0.06277115, 0.05500365, 0.05098187, 0.04446878,
       0.04125288, 0.03193887, 0.02843412])

In [24]:
x_train_pca, x_test_pca, y_train_pca, y_test_pca = train_test_split(x_pca, output, test_size=0.2, random_state=30)

In [27]:
scores1 = []

for model_name in model_params:
    clf = GridSearchCV(model_params[model_name]['model'], model_params[model_name]['params'], cv=5, return_train_score=False)
    clf.fit(x_train_pca, y_train_pca)
    scores1.append({
        'model': model_name,
        'best_score': clf.best_score_,
        'best_params': clf.best_params_
    })

In [28]:
import pandas as pd
df = pd.DataFrame(scores1, columns=['model', 'best_score', 'best_params'])
df

,model,best_score,best_params
0,svm,0.873320,"{'C': 1, 'kernel': 'rbf'}"
1,lr,0.859706,"{'C': 1, 'solver': 'liblinear'}"
2,dt,0.787503,{'criterion': 'gini'}
3,rf,0.871950,{'n_estimators': 100}
